In [7]:
SEASON = '2020-21'

In [8]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = c.MIN_GAMES[SEASON]

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['zTotals'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK'])
p_df['zWS'] = zscore(p_df['WS'])
p_df['zVORP'] = zscore(p_df['VORP'])
p_df['zBPM'] = zscore(p_df['BPM'])
p_df['zTmsWins'] = zscore(p_df['TmsWins'])
p_df['RawScore'] = p_df['zTotals'] + p_df['zWS'] + p_df['zVORP'] + p_df['zBPM'] + p_df['zTmsWins']
p_df['PlayerScore'] = p_df['RawScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,DBPM,BPM,VORP,TmsWins,zTotals,zWS,zVORP,zBPM,zTmsWins,RawScore
1,Nikola Jokić,DEN,C,140.583286,25,72,72,34.6,10.2,18.0,...,3.0,12.1,8.8,47.0,4.371178,5.203555,5.979297,4.334228,1.148572,21.036831
2,Giannis Antetokounmpo,MIL,PF,97.055880,26,61,61,33.0,10.3,18.0,...,2.8,9.0,5.6,46.0,4.355279,2.923639,3.569156,3.233944,1.037366,15.119383
3,Stephen Curry,GSW,PG,85.667920,32,63,63,34.2,10.4,21.7,...,0.4,8.7,5.8,39.0,3.910088,2.416991,3.719790,3.127465,0.258919,13.433252
4,Joel Embiid,PHI,C,79.448466,26,51,51,31.1,9.0,17.6,...,1.2,7.5,3.8,49.0,3.862388,2.332549,2.213452,2.701548,1.370986,12.480924
5,Damian Lillard,POR,PG,79.200836,30,67,67,35.8,9.0,19.9,...,-1.3,6.3,5.0,42.0,3.448997,3.008080,3.117255,2.275632,0.592539,12.442502
6,Luka Dončić,DAL,PG,77.690018,21,66,66,34.3,9.8,20.5,...,0.7,6.8,5.1,42.0,4.100884,1.868122,3.192572,2.453097,0.592539,12.207213
7,Kawhi Leonard,LAC,SF,73.626125,29,52,52,34.1,8.9,17.5,...,1.3,7.3,4.2,47.0,2.940207,2.332549,2.514720,2.630562,1.148572,11.566610
8,Jimmy Butler,MIA,SF,70.843447,31,52,52,33.6,7.0,14.2,...,2.3,7.7,4.3,40.0,2.844809,2.543653,2.590037,2.772534,0.370125,11.121158
9,Rudy Gobert,UTA,C,70.434335,28,71,71,30.8,5.5,8.2,...,2.8,4.9,3.8,52.0,1.970326,3.388066,2.213452,1.778729,1.704606,11.055179
10,LeBron James,LAL,PG,65.387851,36,45,45,33.4,9.4,18.3,...,2.3,8.1,3.8,42.0,3.528495,0.981488,2.213452,2.914507,0.592539,10.230480
